In [87]:
import pandas as pd
from math import isnan
from utils.quack import Quack
pd.set_option("display.precision", 3)
pd.set_option("display.max_columns", 30)

In [88]:
from utils.betting import (
	pivot_lines,
	value_count_plus,
	get_pp_close,
)

In [89]:
pp = Quack.query("select * from pp_close where league_name = 'NFL' ", db_path='C:/Github/NBA_betting_model/databases/lines.duckdb')
pp['alt_line'] = pp['alt_line'].fillna('standard')
pp['date'] = pd.to_datetime(pp['date'], utc=True).dt.tz_convert(None).dt.round('d')

In [90]:
pp

,player,team,line,alt_line,stat,opp,league_id,league_name,event_time,pp_player_id,date,scrape_time,prop_id,event_id,__index_level_0__
0,Jalen Hurts,PHI,174.5,goblin,Pass Yards,GB,9,NFL,2025-11-11 01:00:00+00,206300,2025-11-10,2025-11-08 21:40:00+00,32a9240a-4,dee8d76a-3,30463.0
1,Jordan Love,GB,14.5,demon,Longest Rush,PHI,9,NFL,2025-11-11 01:00:00+00,206301,2025-11-10,2025-11-08 21:40:00+00,9a2c3382-8,dee8d76a-3,30713.0
2,Christian Watson,GB,24.5,standard,Receiving Yards in First 2 Receptions,PHI,9,NFL,2025-11-11 01:00:00+00,210337,2025-11-10,2025-11-08 21:40:00+00,134434ed-6,dee8d76a-3,30389.0
3,A.J. Brown,PHI,22.5,standard,Receiving Yards in First 2 Receptions,GB,9,NFL,2025-11-11 01:00:00+00,206304,2025-11-10,2025-11-08 21:40:00+00,7b3bf1e1-d,dee8d76a-3,30390.0
4,Saquon Barkley,PHI,21.5,standard,Rush Yards in First 5 Attempts,GB,9,NFL,2025-11-11 01:00:00+00,206302,2025-11-10,2025-11-08 21:40:00+00,fed873d5-2,dee8d76a-3,30391.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118157,Najee Harris,PIT,1.5,standard,Receptions,CIN,9,NFL,2023-12-23 15:00:00-06:00,150691,2023-12-24,2023-12-23 15:02:00-06:00,1c69cb3a-c,918f5321-2,31.0
118158,Najee Harris,PIT,53.5,standard,Rush Yards,CIN,9,NFL,2023-12-23 15:00:00-06:00,150691,2023-12-24,2023-12-23 15:02:00-06:00,d961345d-9,918f5321-2,32.0
118159,Najee Harris,PIT,39.5,goblin,Rush Yards,CIN,9,NFL,2023-12-23 15:00:00-06:00,150691,2023-12-24,2023-12-23 15:02:00-06:00,d961345d-9,918f5321-2,33.0
118160,Najee Harris,PIT,8.5,standard,Receiving Yards,CIN,9,NFL,2023-12-23 15:00:00-06:00,150691,2023-12-24,2023-12-23 15:02:00-06:00,6dc91603-1,918f5321-2,34.0


In [91]:
Quack.query("""select * from weekly
             where pat_made > 0 and season = 2025 limit 5""")

,game_id,gameday,player_id,player_name,player_display_name,position,position_group,headshot_url,season,week,season_type,team,opponent_team,completions,attempts,...,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance,fantasy_points,fantasy_points_ppr
0,2025_01_CIN_CLE,2025-09-07,00-0036854,E.McPherson,Evan McPherson,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,1,REG,CIN,CLE,0,0,...,35,0,0,2,2,0,0,1.0,0,0,0,0,0,0.0,0.0
1,2025_02_SF_NO,2025-09-14,00-0038905,B.Grupe,Blake Grupe,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,2,REG,NO,SF,0,0,...,0,40,0,3,3,0,0,1.0,0,0,0,0,0,0.0,0.0
2,2025_10_LV_DEN,2025-11-06,00-0034161,D.Carlson,Daniel Carlson,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,10,REG,LV,DEN,0,0,...,0,48,0,1,1,0,0,1.0,0,0,0,0,0,0.0,0.0
3,2025_03_ATL_CAR,2025-09-21,00-0040279,R.Fitzgerald,Ryan Fitzgerald,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,3,REG,CAR,ATL,0,0,...,133,0,0,3,3,0,0,1.0,0,0,0,0,0,0.0,0.0
4,2025_10_LV_DEN,2025-11-06,00-0032569,W.Lutz,Wil Lutz,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,10,REG,DEN,LV,0,0,...,32,59,0,1,1,0,0,1.0,0,0,0,0,0,0.0,0.0


In [92]:
with open("./data/queries/pp_new_stats.sql", "r") as f:
    sql = f.read()

games = Quack.query(sql)

In [93]:
pp2nfl = {
    "FG": "fg_made",
    "INT": "passing_interceptions",
    "PAT Made": "pat_made",
    "Pass Attempts": "attempts",
    "Pass Completions": "completions",
    "Pass Deflections": "def_pass_defended",
    "Pass TDs": "passing_tds",
    "Pass Yards": "passing_yards",
    "Rec Targets": "targets",
    "Receiving Yards": "receiving_yards",
    "Receptions": "receptions",
    "Rush TDs": "rushing_tds",
    "Rush Yards": "rushing_yards",
    "Sacks": "def_sacks",
    "Sacks Taken": "sacks_suffered",
    "Receiving Yards in First 2 Receptions": "yards_2_catches",
    "Rush Yards in First 5 Attempts": "yards_5_rushes",
    "Quarters With 5+ Rush Yards": "qtrs_over_5_rush_yards",
    "Quarters With 1+ Reception": "qtrs_with_reception",
    "Longest Rush": "longest_rush",
    "Longest Reception": "longest_reception",
    "Rush Attempts": "carries",
    "Pass+Rush Yds": "passing_yards+rushing_yards",
    "Pass+Rush+Rec TDs": "passing_tds+rushing_tds+receiving_tds",
    "AST": "def_tackle_assists",
    "Yards on First Rush Attempt": "yards_first_rush",
    "Tackles For Loss": "def_tackles_for_loss",
    "Yards on First Reception": "yards_first_catch",
    "Quarters with 5+ Rush Yards": "qtrs_over_5_rush_yards",
    "Quarters with 10+ Receiving Yards": "qtrs_over_10_rec_yards",
}

In [94]:
pp['stat_nfl'] = pp['stat'].apply(lambda x: pp2nfl.get(x,x))
games = games.rename({'player_display_name':'player', 'opponent_team':'opp', 'gameday':'date'},axis=1)
games['date'] = pd.to_datetime(games['date'])

In [95]:
games['game_line'] = games.apply(lambda x: x['spread_line'] if x['home_team'] == x['team'] else -x['spread_line'],axis=1)
games['Rush+Rec TDs'] = games['receiving_tds'] + games['rushing_tds']
games['Completion Percentage'] = 100 * (games['completions'] / games['attempts'])
games['Pass+Rush Yds'] = games['passing_yards'] + games['rushing_yards']
games['Rush+Rec Yds'] = games['receiving_yards'] + games['rushing_yards']
games['Rush Yards Per Carry'] = games['rushing_yards'] / games['carries']
games['Kicking Points'] = games['pat_made'] + (3 * games['fg_made'])
games['Fantasy Score'] = ((games['passing_yards'] * 0.04) + (games['passing_tds'] * 4) + (2 * -games['passing_interceptions']) + (games['rushing_yards'] * 0.1) + (games['receiving_yards'] * 0.1) + #missing 2 pt conv
                    (games['receptions']) + (2 * (-games['receiving_fumbles_lost'] - games['rushing_fumbles_lost'] )) + (games['Rush+Rec TDs'] * 6) + 
                    games['pat_made'] + (3 * games['fg_made']) + games['fg_made_40_49'] + (games['fg_made_50_59'] * 2) + (games['fg_made_60_'] * 3) - games['pat_missed']  - games['fg_missed'] )

In [96]:
ref2pp = {
    'GNB':'GB',
    'JAX':'JAC',
    'KAN':'KC',
    'LVR':'LV',
    'NOR':'NO',
    'NWE':'NE',
    'SFO':'SF',
    'TAM':'TB',
}
games['team'] = games['team'].apply(lambda x: ref2pp.get(x,x))


def get_results(lines, games):
	def calc_outcome(row):
		outcome = 0
		for part in row['stat_nfl'].split('+'):
			try:
				outcome += row[part]
			except KeyError:
				return float('nan')
		return outcome

	comb = lines.merge(games, on=['player', 'date', 'team', 'opp'])
	comb['result'] = comb.apply(lambda row: calc_outcome(row), axis=1)
	comb['o_u'] = [
		'under' if x > y else 'over' if x < y else 'push' if x == y else 'N/A'
		for x, y in zip(comb['line'], comb['result'])
	]
	return comb.loc[comb.o_u != 'N/A'].reset_index(drop=True)

df = get_results(pp, games)
df.sample(3)

,player,team,line,alt_line,stat,opp,league_id,league_name,event_time,pp_player_id,date,scrape_time,prop_id,event_id,__index_level_0__,...,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium,game_line,Rush+Rec TDs,Completion Percentage,Pass+Rush Yds,Rush+Rec Yds,Rush Yards Per Carry,Kicking Points,Fantasy Score,o_u
14219,Alex Singleton,DEN,0.5,demon,Sacks,NYG,9,NFL,2025-10-19 20:00:00+00,260189,2025-10-19,2025-10-19 19:40:00+00,a9d55593-f,395348e4-3,666.0,...,Bo Nix,Brian Daboll,Sean Payton,Brad Allen,DEN00,Empower Field at Mile High,7.5,0,NaN,0,0,NaN,0,0.0,under
14953,Kenneth Gainwell,PIT,24.5,demon,Receiving Yards,CIN,9,NFL,2025-10-17 00:00:00+00,250224,2025-10-16,2025-10-17 00:10:00+00,521912f8-c,ddb9093e-d,517.0,...,Joe Flacco,Mike Tomlin,Zac Taylor,Clay Martin,CIN00,Paycor Stadium,5.5,0,NaN,14,15,4.667,0,3.5,under
74780,Jerry Jeudy,CLE,3.5,standard,Receptions,LV,9,NFL,2024-09-29 20:00:00+00,210873,2024-09-29,2024-09-29 20:10:00+00,5aa4d57d-6,34c4366e-2,790.0,...,Gardner Minshew,Kevin Stefanski,Antonio Pierce,Brad Allen,VEG00,Allegiant Stadium,2.5,0,NaN,0,72,NaN,0,13.2,over


In [130]:
from math import floor


def get_tiers(value):
    if isnan(value):
        return value
    else:
        return floor(value / 10) * 10


def tiers(value):
    if isnan(value):
        return value
    if value <= -4.5:
        return "big L"
    if value <= 4.5:
        return "even"
    else:
        return "big W"


df["favored"] = df["game_line"].apply(lambda x: tiers(x))
# df['bucket'] = df['yard_pct'].apply(lambda x: get_tiers(x))
subset = df.loc[(df.alt_line != "goblin") & (df["date"] > "2025-10-01")].copy()
result = pivot_lines(subset, ["alt_line", "stat", "favored"]).sort_values(
    by="over_ratio", ascending=False
)
result.loc[(result["count"] > 15) & (result["alt_line"] == "standard")].sort_values(
    by=["over_ratio"], ascending=False
).drop(["over", "push", "under"], axis=1).tail(25)

o_u,alt_line,stat,favored,over_ratio,count
127,standard,Rush Yards in First 5 Attempts,big W,0.489,37.0
110,standard,Rec Targets,even,0.487,243.0
66,standard,Completion Percentage,big L,0.485,23.0
78,standard,Kicking Points,big L,0.477,35.0
85,standard,Longest Rush,big W,0.476,74.0
95,standard,Pass Completions,even,0.474,69.0
117,standard,Receptions,big L,0.473,86.0
72,standard,Fantasy Score,big L,0.471,196.0
86,standard,Longest Rush,even,0.471,177.0
128,standard,Rush Yards in First 5 Attempts,even,0.469,72.0


In [131]:
df['stat'].value_counts()

stat
Receiving Yards                          19807
Receptions                               10647
Rush Yards                                9343
Longest Reception                         6962
Pass Yards                                4367
Fantasy Score                             4043
Rush Attempts                             3248
Sacks                                     2878
Longest Rush                              2715
Rec Targets                               2469
Pass+Rush Yds                             2071
Pass TDs                                  1570
Pass Attempts                             1483
FG                                        1471
Pass Completions                          1420
INT                                       1144
Kicking Points                             855
Yards on First Reception                   763
Receiving Yards in First 2 Receptions      701
Rush Yards in First 5 Attempts             528
PAT Made                                   444
Yards on

In [132]:
res = pivot_lines(df.loc[df.date > '2025-10-01'], ['stat_nfl', 'alt_line', 'favored']).sort_values(by='over_ratio').copy()
res.loc[res.stat_nfl=='carries']

o_u,stat_nfl,alt_line,favored,over,push,under,over_ratio,count
24,carries,demon,big L,24.0,0.0,96.0,0.204,120.0
25,carries,demon,big W,28.0,0.0,101.0,0.219,129.0
26,carries,demon,even,62.0,0.0,219.0,0.222,281.0
30,carries,standard,big L,15.0,1.0,30.0,0.364,46.0
31,carries,standard,big W,23.0,4.0,34.0,0.418,61.0
32,carries,standard,even,47.0,4.0,62.0,0.437,113.0
28,carries,goblin,big W,59.0,0.0,43.0,0.585,102.0
27,carries,goblin,big L,49.0,0.0,35.0,0.590,84.0
29,carries,goblin,even,122.0,0.0,83.0,0.598,205.0


In [133]:
working = list(res['stat_nfl'].unique())
available = list(pp.loc[pp.date > '2025-10-01', 'stat_nfl'].unique())
[x for x in available if x not in working and 'Combo' not in x]

['Completions in First 10 Pass Attempts',
 'Yards on First Pass Completion',
 'Rush+Rec Yds',
 'Longest Completion',
 'Punts',
 'Quarters with 1+ Reception',
 'Rush+Rec TDs',
 'Halves with 25+ Rush Yards',
 'Tackles+Ast',
 'Quarters with 30+ Pass Yards',
 'Quarters with 5+ Receiving Yards',
 'Quarters with 10+ Rush Yards',
 'Halves with 100+ Pass Yards',
 'Halves with 25+ Receiving Yards',
 'Halves with 150+ Pass Yards',
 'Halves with 40+ Receiving Yards',
 'Halves with 50+ Rush Yards',
 'Quarters with 25+ Pass Yards',
 'Quarters with 20+ Pass Yards',
 'Quarters with 15+ Receiving Yards',
 'qtrs_over_10_rec_yards',
 'Quarters with 2+ Receptions',
 'Passing Yards in First 10 Attempts',
 'Halves with 50+ Receiving Yards',
 'Quarters with 50+ Pass Yards',
 'Quarters with 20+ Rush Yards',
 'Quarters with 15+ Rush Yards']

In [134]:
schedule = Quack.query('select * from schedule where season = 2025')

In [136]:
result

o_u,alt_line,stat,favored,over,push,under,over_ratio,count
82,standard,Longest Reception,big W,118.0,1.0,65.0,0.637,184.0
71,standard,FG,even,17.0,0.0,10.0,0.595,27.0
114,standard,Receiving Yards in First 2 Receptions,big L,37.0,0.0,24.0,0.592,61.0
97,standard,Pass TDs,big W,5.0,0.0,2.0,0.588,7.0
83,standard,Longest Reception,even,228.0,0.0,160.0,0.585,388.0
...,...,...,...,...,...,...,...,...
19,demon,PAT Made,big W,4.0,0.0,22.0,0.181,26.0
31,demon,Pass Yards,big W,21.0,0.0,100.0,0.179,121.0
38,demon,Quarters with 5+ Rush Yards,even,9.0,0.0,47.0,0.174,56.0
6,demon,INT,big L,6.0,0.0,35.0,0.167,41.0


In [144]:
today = pd.concat((
    pp.merge(schedule, left_on=['team', 'opp'], right_on=['home_team', 'away_team']),
    pp.merge(schedule, left_on=['team', 'opp'], right_on=['away_team', 'home_team'])
))
today = today.loc[today.week==10]
today['game_line'] = today.apply(lambda x: x['spread_line'] if x['home_team'] == x['team'] else -x['spread_line'],axis=1)
today['favored'] = today['game_line'].apply(lambda x: tiers(x))
today = today[['player','team','stat','line','alt_line','game_line','favored']].merge(result, on=['stat','alt_line', 'favored'])
tst = today.loc[today.alt_line=='standard'].sort_values(by='over_ratio').reset_index(drop=True) #.iloc[50:100]
tst.groupby(['favored','stat','alt_line']).head(5).iloc[0:40]

,player,team,stat,line,alt_line,game_line,favored,over,push,under,over_ratio,count
0,Jake Bates,DET,Kicking Points,7.50,standard,8.5,big W,8.0,1.0,21.0,0.333,30.0
1,Michael Badgley,IND,Kicking Points,7.50,standard,6.5,big W,8.0,1.0,21.0,0.333,30.0
2,Jason Myers,SEA,Kicking Points,6.50,standard,7.0,big W,8.0,1.0,21.0,0.333,30.0
3,Wil Lutz,DEN,Kicking Points,7.50,standard,9.5,big W,8.0,1.0,21.0,0.333,30.0
4,Eddy Pineiro,CAR,Kicking Points,5.50,standard,5.5,big W,8.0,1.0,21.0,0.333,30.0
22,Chris Rodriguez,WAS,Longest Rush,9.50,standard,-8.5,big L,26.0,0.0,52.0,0.352,78.0
23,Kyler Murray,ARI,Longest Rush,11.50,standard,-7.0,big L,26.0,0.0,52.0,0.352,78.0
24,Michael Penix,ATL,Longest Rush,5.50,standard,-6.5,big L,26.0,0.0,52.0,0.352,78.0
25,Jacory Croskey-Merritt,WAS,Longest Rush,10.50,standard,-8.5,big L,26.0,0.0,52.0,0.352,78.0
26,Geno Smith,LV,Longest Rush,6.50,standard,-9.5,big L,26.0,0.0,52.0,0.352,78.0


In [105]:
Quack.query('SHOW TABLES')

,name
0,depth
1,ids
2,opp_feats
3,pbp
4,player_weekly_agg
5,rosters
6,rusher_adv
7,schedule
8,snaps
9,team_feats


In [106]:
today.loc[today.alt_line=='standard'].sort_values(by='over_ratio').reset_index(drop=True).head(30)

,player,team,stat,line,alt_line,game_line,over,push,under,over_ratio,count
0,Bijan Robinson,ATL,Quarters with 5+ Rush Yards,3.5,standard,-3.5,12.0,0.0,22.0,0.386,34.0
1,James Conner,ARI,Quarters with 5+ Rush Yards,3.5,standard,7.0,12.0,0.0,22.0,0.386,34.0
2,Ashton Jeanty,LV,Quarters with 5+ Rush Yards,3.5,standard,-3.5,12.0,0.0,22.0,0.386,34.0
3,De'Von Achane,MIA,Quarters with 5+ Rush Yards,3.5,standard,1.5,12.0,0.0,22.0,0.386,34.0
4,Tony Pollard,TEN,Quarters with 5+ Rush Yards,3.5,standard,-5.5,12.0,0.0,22.0,0.386,34.0
5,Bucky Irving,TB,Quarters with 5+ Rush Yards,3.5,standard,-2.5,12.0,0.0,22.0,0.386,34.0
6,Jahmyr Gibbs,DET,Quarters with 5+ Rush Yards,3.5,standard,6.0,12.0,0.0,22.0,0.386,34.0
7,Kyren Williams,LA,Quarters with 5+ Rush Yards,3.5,standard,5.5,12.0,0.0,22.0,0.386,34.0
8,Russell Wilson,NYG,Rush Attempts,4.0,standard,-4.5,85.0,9.0,126.0,0.407,220.0
9,Lamar Jackson,BAL,Rush Attempts,9.5,standard,12.5,85.0,9.0,126.0,0.407,220.0


In [107]:
today.loc[(today.alt_line=='demon') & (today.stat != 'Sacks')].sort_values(by='over_ratio').tail(20)

,player,team,stat,line,alt_line,game_line,over,push,under,over_ratio,count
3661,Saquon Barkley,PHI,Longest Reception,19.5,demon,1.5,400.0,0.0,841.0,0.322,1241.0
3667,A.J. Brown,PHI,Longest Reception,29.5,demon,1.5,400.0,0.0,841.0,0.322,1241.0
3668,A.J. Brown,PHI,Longest Reception,34.5,demon,1.5,400.0,0.0,841.0,0.322,1241.0
3672,DeVonta Smith,PHI,Longest Reception,24.5,demon,1.5,400.0,0.0,841.0,0.322,1241.0
3673,DeVonta Smith,PHI,Longest Reception,29.5,demon,1.5,400.0,0.0,841.0,0.322,1241.0
3934,Marvin Mims Jr.,DEN,Longest Reception,29.5,demon,2.5,400.0,0.0,841.0,0.322,1241.0
3271,Rachaad White,TB,Longest Reception,9.5,demon,-2.5,400.0,0.0,841.0,0.322,1241.0
87,Jayden Higgins,HOU,Longest Reception,24.5,demon,2.5,400.0,0.0,841.0,0.322,1241.0
71,Nico Collins,HOU,Longest Reception,29.5,demon,2.5,400.0,0.0,841.0,0.322,1241.0
63,Nico Collins,HOU,Longest Reception,34.5,demon,2.5,400.0,0.0,841.0,0.322,1241.0


In [108]:
result.loc[(result['count'] > 10) &(result['alt_line']=='demon')].sort_values(by=['over_ratio'],ascending=False).drop(['over','push','under'],axis=1).head(30)

o_u,alt_line,stat,over_ratio,count
19,demon,Sacks Taken,0.369,32.0
4,demon,Longest Reception,0.322,1241.0
8,demon,Pass Completions,0.316,115.0
9,demon,Pass TDs,0.289,210.0
6,demon,PAT Made,0.281,120.0
11,demon,Pass+Rush Yds,0.267,340.0
3,demon,Kicking Points,0.246,61.0
18,demon,Sacks,0.244,1112.0
7,demon,Pass Attempts,0.243,124.0
10,demon,Pass Yards,0.239,502.0


In [109]:
result = pivot_lines(subset, ['alt_line', 'stat', 'pos']).sort_values(by='over_ratio', ascending=False)
result.loc[(result.alt_line == 'standard') & (result['count'] >= 15)].sort_values(by=['over_ratio'],ascending=False).tail(20)

KeyError: 'pos'

In [ ]:
def get_corr(one, two,three=pd.DataFrame(), grouper=[], relation='same'):
    df1 = one.copy(deep=True)
    df2 = two.copy(deep=True)
    df3 = three.copy(deep=True)
    #get cols to merge on
    grouper = ['alt_line','stat'] + grouper
    home_cols = ['team', 'date']
    opp_cols = ['opp', 'date']
    if relation=='same':
        right_cols = home_cols
    else:
        right_cols = opp_cols

    #fixing weird column renaming
    if not df3.empty:
        a = 4
    else:
        a = 3
    count_cols = []
    o_u_cols = []
    for x in grouper:
        for n in range(1,a):
            count_cols += [x+f'_{n}']
            o_u_cols += [f'o_u_{n}']
    o_u_cols = sorted(list(set(o_u_cols)))

    raw_cols = list(set(grouper + home_cols + right_cols + ['o_u'] + ['player']))
    
    #merge dfs
    comb_corr = df1.loc[(df1.alt_line!='goblin') & (df1.o_u != 'push'), raw_cols].merge(
        df2.loc[(df2.alt_line!='goblin') & (df2.o_u != 'push'), raw_cols],
        left_on=home_cols,
        right_on=right_cols,
        suffixes=('_1', '_2'),
        )
    #filter out same player corr and mirrored lines
    comb_corr = comb_corr.loc[(comb_corr.player_1 != comb_corr.player_2) & 
                              ~((comb_corr.player_1 < comb_corr.player_2) & (comb_corr.stat_1 == comb_corr.stat_2))
                              ].reset_index(drop=True)
    if not df3.empty:
        comb_corr = comb_corr.merge(
            df3.loc[(df3.alt_line!='goblin'), raw_cols],
            left_on=home_cols,
            right_on=right_cols,
            #suffixes=('', '_2'),
            )
        comb_corr.columns = [x+'_3' if (x in raw_cols) else x for x in comb_corr.columns]
        comb_corr = comb_corr.loc[comb_corr.player_1 != comb_corr.player_3].reset_index(drop=True)
        comb_corr = comb_corr.loc[comb_corr.player_2 != comb_corr.player_3].reset_index(drop=True)
    corr = value_count_plus(
        comb_corr,
        count_cols,
        o_u_cols,
    )
    corr['relation'] = relation
    corr = corr.loc[~((corr.alt_line_1.isin(['demon', 'goblin'])) & (corr.o_u_1 == 'under'))]
    corr = corr.loc[~((corr.alt_line_2.isin(['demon', 'goblin'])) & (corr.o_u_2 == 'under'))]
    if not df3.empty:
        corr = corr.loc[~((corr.alt_line_3.isin(['demon', 'goblin'])) & (corr.o_u_3 == 'under'))]

    corr = corr.loc[corr['count'] > 0].sort_values(by='proportion', ascending=False)
    corr['equiv'] = corr['proportion'] ** (1/(a-1))
    corr = corr.rename({'count':'count_all'},axis=1)

    #filtering extraneous permutations
    numbered_cols = []
    for num in range(1,a):
        numbered_cols.append(num)
        corr[num] = ''
        for col in (['stat', 'alt_line', 'o_u'] + grouper):
            num_col = f'{col}_{num}'
            corr[num] += corr[num_col].astype(str)
    if a == 4:
        corr['concat'] = corr.apply(lambda x: str(sorted(list((x[1],x[2],x[3])))),axis=1)
    else:
        corr['concat'] = corr.apply(lambda x: str(sorted(list((x[1],x[2])))),axis=1)
    corr = corr.groupby(['concat']).head(1)
    corr = corr.sort_values(by='equiv',ascending=False).drop([1,2,3,'concat'],axis=1,errors='ignore')

    return corr.sort_values(by='proportion',ascending=False)

def weight_corr(corr, hit_df,suff):
    merge_cols = []
    for col in hit_df.columns:
        if 'hit_' in col:
            hit_col = col
        elif col != ('count') and col != ('proportion'):
            merge_cols.append(col)
    for n in range(1,4):
        n_merge_cols = [col + f'_{n}' for col in merge_cols]
        if f'alt_line_{n}' in corr.columns:
            corr = corr.merge(
                hit_df,
                left_on=n_merge_cols,
                right_on=merge_cols,
                how='left',
                suffixes=[f'_{n-1}',f'_{n}']
            )
            corr.drop(merge_cols, axis=1, inplace=True)
            corr.rename({f'hit_{suff}': f'hit_{suff}_{n}'},axis=1,inplace=True)
    corr['expected'] = 1
    for n in range(1,4):
        col = f'hit_{suff}_{n}'
        if col in corr.columns:
            corr['expected'] = corr['expected'] * corr[col]
    corr['diff'] = corr['proportion'] - corr['expected']
    return corr

hit_stat = value_count_plus(
	df.loc[df.o_u.isin(['under', 'over'])], ['alt_line', 'stat'], ['o_u']
)
#hit_stat['hit_stat'] = hit_stat.apply(lambda x: pad_results(x), axis=1)
hit_stat = hit_stat.rename({'proportion':'hit_stat'},axis=1)
hit_stat.head(3)



,alt_line,stat,o_u,hit_stat,count
0,demon,AST,over,0.500,2
1,demon,AST,under,0.500,2
3,demon,FG,over,0.258,203


In [ ]:
corr = get_corr(df, df,df,  relation='same', grouper=['pos'])
corr = weight_corr(corr,hit_stat,'stat')
corr.loc[(corr.count_all > 55)].head(25)

In [ ]:
corr.loc[(corr.count_all > 55)].head(25)

In [ ]:
corr.loc[((
    (corr.pos_1=='QB') &
    (corr.pos_2=='K')) | 
    (
    (corr.pos_2=='QB') &
    (corr.pos_1=='K')
) )& (corr.count_all > 16)
].sort_values(by='diff',ascending=False).head(20)

In [ ]:
corr2 = get_corr(df, df, relation='same', grouper=['FantPos'])

In [ ]:
corr2.loc[(corr2.count_all > 15) & (corr2.stat_1=='FG') & (corr2.stat_2=='Pass Attempts')].head(25)

In [ ]:
corr2.loc[(corr2.stat_1=='Pass Yards') & (corr2.stat_2=='Fantasy Score') & (corr2.stat_3=='FG')]

In [ ]:
opp = get_corr(df, df, relation='opp',grouper=['FantPos'])
opp = opp.loc[(opp.count_all > 20) & ((opp.FantPos_1 =='QB') | (opp.FantPos_2 == 'QB'))]
opp.loc[(opp.count_all > 35)].head(20)

In [ ]:
wg = weight_corr(opp,hit_stat,'stat')
wg['diff'] = wg['proportion'] - wg['expected']
wg.loc[(wg.count_all > 80) & (wg.alt_line_1=='demon')].sort_values(by='diff',ascending=False).head(30)

In [ ]:
corr = get_corr(df, df, relation='same',grouper=['FantPos'])
corr.loc[(corr.count_all > 40) & (corr.stat_1 >= corr.stat_2)].head(30)
#corr.loc[(corr.count_all > 20) & (corr.stat_1 >= corr.stat_2) & (corr.alt_line_1 == 'demon')].head(30)

In [ ]:
corr2.loc[(corr2.stat_1.str.contains('TD')) & (corr2.stat_3.str.contains('TD')) & (corr2.FantPos_2 == 'K')]

In [ ]:
corr.loc[(corr.count_all > 20)& ((corr.alt_line_1 == 'demon') | (corr.alt_line_2 == 'demon'))].head(15)

In [ ]:
corr2.loc[(corr2.count_all > 25) & (
    (corr2.alt_line_1 == 'demon') |
    (corr2.alt_line_2 == 'demon') |
    (corr2.alt_line_3 == 'demon')) & (corr2.stat_1.str.contains("Fantasy")
)].head(30)

In [ ]:
corr2.loc[corr2.stat_1.str.contains('TD') & (corr2.count_all > 10)]

In [ ]:
corr2.to_csv('../reference_data/NFL/corr/corr3.csv')

In [ ]:
corr2.loc[(corr2.count_all > 35)].head(30)

In [ ]:
base = df[['player','team','line','alt_line','stat','opp','event_time','o_u']]
base

In [ ]:
pair =base.merge(base,on=['team','opp','event_time'])
pair = pair.loc[~(pair.player_x <= pair.player_y)]
pair.loc[(pair.player_x=='Dak Prescott') & (pair.player_y.str.contains('Lamb')) & (pair.stat_x=='Pass Yards') & (pair.stat_y=='Receiving Yards'),['o_u_x','o_u_y']].value_counts().reset_index()

In [ ]:
dk = pd.read_parquet('../Lines/dk/dk_2024_10_5.parquet')
dk.loc[dk['player'].str.contains('Chief',na=False)].head(5)

In [ ]:
play_corr = pair.groupby(['player_x','player_y','stat_x','stat_y','alt_line_x','alt_line_y'])[['o_u_x','o_u_y']].value_counts().reset_index()

In [ ]:
lovers = value_count_plus(
        play_corr.loc[play_corr.alt_line_x=='standard'],
        ['player_x','player_y','stat_x','stat_y','alt_line_x','alt_line_y'],
        ['o_u_x','o_u_y'],
    )

In [ ]:
lovers.loc[(lovers.player_x=='Dak Prescott') & (lovers.player_y.str.contains('Lamb')) & (lovers.stat_x=='Pass Yards') & (lovers.stat_y=='Receiving Yards')]

In [ ]:
pair.loc[(pair.player_x=='Dak Prescott') & (pair.player_y.str.contains('Lamb')) & (pair.stat_x=='Pass Yards') & (pair.stat_y=='Receiving Yards') & 
         (pair.alt_line_x=='standard') & (pair.alt_line_y=='standard')]

In [ ]:
common_cols = [ 'team', 'opp', 'home', 'roof', 'surface', 'weather', 'game_line', 'favored', 'total',
       'season',  'date']
qb_cols =     ['player', 'Pass Completions', 'Pass Attempts', 'Pass Yards',
       'Pass TDs', 'INT', 'Sacks', 'Yards Sacked', 'Longest Pass',
       'Passing Rate',  'snaps', 'pct', 'pos']
wr_cols =     ['player', 'Rec Targets', 'Receptions', 'Receiving Yards', 'Rush+Rec Yds', 
       'Rec TDs','Rush+Rec TDs', 'Longest Reception', 'Fantasy Score',  'snaps', 'pct', 'pos']
rb_cols =     ['player', 'Rush Attempts', 'Rush Yards', 'Rush TDs', 'Rush+Rec Yds', 
       'Longest Rush', 'Rec Targets', 'Receptions', 'Receiving Yards',
       'Rec TDs','Rush+Rec TDs', 'Longest Reception', 'Fumbles', 'Fumbles Lost', 'Fantasy Score', 'snaps', 'pct',
        'pos']

k_cols = ['player', 'XPM','Kicking Points', 'Fantasy Score', 'FG','snaps', 'pct', 'pos']

small = games.loc[games['date'] > '2014-09-01']

qb = small.loc[small.pos=='QB'][
    qb_cols + common_cols
]
wr = small.loc[small.pos=='WR'][
    wr_cols + common_cols
]
rb = small.loc[small.pos=='RB'][
    rb_cols + common_cols
]
te = small.loc[small.pos=='TE'][
    wr_cols + common_cols
]
k = small.loc[small.pos=='K'][
    k_cols + common_cols
]

In [ ]:
p1 = 'Jared Goff'
b = small.loc[small.player==p1]

p2 = 'Amon-Ra St. Brown'
t = small.loc[small.player==p2]

both = b.merge(t, on=common_cols)
print(len(both))
both.columns

In [ ]:
import matplotlib.pyplot as plt
stat = 'Pass Attempts'
stat1 = stat + '_x'
stat2 = stat + '_y'
stat2 = 'Rec Targets_y'

plt.axvline(x=both[stat1].median(), label=f'{p1} avg', color='r')
plt.axhline(y=both[stat2].median(), label=f'{p2} avg')
plt.scatter(both[stat1], both[stat2], c=both.index, cmap=plt.cm.coolwarm)
plt.xlabel(p1)
plt.ylabel(p2)
plt.title(stat)
plt.legend()

In [ ]:
both[[stat1,stat2]]